In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This project read the Google NEWS and get the most frequest word. It can improve the editorial vocabulary. 

In [17]:
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import string
from nltk.corpus import stopwords, words
from nltk.corpus import wordnet as wn
import pandas as pd

In [18]:
news_url = "https://news.google.com/rss"
Client = urlopen(news_url)
xml_page = Client.read()
Client.close()

soup_page = soup(xml_page, "xml")
news_list = soup_page.findAll("item")

newslist = []

for news in news_list:
    cleantext = soup(news.description.text, "lxml").text
    newslist.append(cleantext)

In [19]:
newslist[1]

"President Trump Has Temporarily Boosted Unemployment -- but There Are Strings Attached\xa0\xa0The Motley FoolBreaking down the executive actions Trump signed on coronavirus relief\xa0\xa0CNNCould You Get A Second Stimulus Check By Executive Order?\xa0\xa0ForbesLiz Peek: Trump's big win on relief orders – here's why Pelosi, Schumer are so unhappy\xa0\xa0Fox NewsTrump’s actions on pandemic relief aren’t illegal. They’re just ineffective.\xa0\xa0The Washington PostView Full Coverage on Google News"

In [20]:
def text_process(title):
    
    nop = [char for char in title if char not in string.punctuation]
    
    nop = ''.join(nop)
    
    return [word for word in nop.split() if word in word.lower() not in stopwords.words('english')]


def word_list(wordlist):
    """"  
     Return only word form the news sentences
    """
    li = []
    for word in wordlist:
        if word in words.words():
            li.append(word)
    return li

In [21]:
finalword = []
for w in newslist:
    word = text_process(w.lower())
    finalword.append(word_list(word))

In [22]:
finalword

[['sweep',
  'magnificent',
  'mile',
  'police',
  'shooting',
  'new',
  'york',
  'post',
  'widespread',
  'caught',
  'video',
  'shooting',
  'fox',
  'chaos',
  'extensive',
  'damage',
  'downtown',
  'night',
  'rioting',
  'mag',
  'mile',
  'press',
  'conference',
  'following',
  'shooting',
  'fox',
  'full',
  'coverage',
  'news'],
 ['president',
  'trump',
  'temporarily',
  'unemployment',
  'attached',
  'motley',
  'executive',
  'trump',
  'relief',
  'get',
  'second',
  'stimulus',
  'check',
  'executive',
  'order',
  'peek',
  'big',
  'win',
  'relief',
  'unhappy',
  'fox',
  'pandemic',
  'relief',
  'illegal',
  'ineffective',
  'full',
  'coverage',
  'news'],
 ['sen',
  'getting',
  'dirt',
  'lawmaker',
  'twisting',
  'fox',
  'say',
  'trump',
  'president',
  'outlandish',
  'increasingly',
  'met',
  'collective',
  'shrug',
  'equal',
  'coverage',
  'balanced',
  'dishonest',
  'come',
  'presidential',
  'history',
  'boston',
  'full',
  'covera

In [23]:
# Make 2D array to 1D Array

from itertools import chain
flatten_list = list(chain.from_iterable(finalword))
length = len(flatten_list)
length

NameError: name 'lenght' is not defined

In [9]:
repeatwordDict = {}
repeatwordList = []
count = 1

for w in flatten_list:
    if w not in repeatwordList:
        repeatwordDict[w] = count
    else:
        repeatwordDict[w] = repeatwordDict[w] + 1
    
print(len(repeatwordDict))

orderDict = sorted(repeatwordDict.items(), key = lambda x: x[1], reverse=True)

501


In [12]:
topwords = []
wordDict = {}

for w in orderDict: 
    w = w[0]
    try:
        syns = wn.synsets(w)
        tmp = wn.synsets(w)[0].pos()
        if tmp == 'a' or tmp == 's': 
            wordDict = {
                'Name': w,
                'Type': 'Adjective',
                'Defination': syns[0].definition(),
                'Example': syns[0].examples()
            }
            topwords.append(wordDict)
        if tmp == 'r':
            wordDict = {
                'Name': w,
                'Type': 'Adverb',
                'Defination': syns[0].definition(),
                'Example': syns[0].examples()
            }
            topwords.append(wordDict)
        if tmp == 'v':
            wordDict = {
                'Name': w,
                'Type': 'Verb',
                'Defination': syns[0].definition(),
                'Example': syns[0].examples()
            }
            topwords.append(wordDict)
    except:
        print('Error')

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [15]:
Engwords = pd.DataFrame(topwords)
Engwords = Engwords.sort_values('Name', ignore_index=True)
Engwords.head()

,Name,Type,Defination,Example
0,abroad,Adjective,in a foreign country,"[markets abroad, overseas markets]"
1,actively,Adverb,in an active manner,[he participated actively in the war]
2,ahead,Adjective,having the leading position or higher score in...,"[he is ahead by a pawn, the leading team in th..."
3,allow,Verb,make it possible through a specific action or ...,"[This permits the water to rush in, This seale..."
4,along,Adverb,with a forward motion,"[we drove along admiring the view, the horse t..."


In [16]:
Engwords.to_csv('words.csv', index=False)